In [6]:
import os
import random
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import cv2
import shutil
#--------- Code Written by Khai Tha ----------------

# Set the seed for reproducibility
ia.seed(1)
random.seed(1)

# Augmentation sequence (adjust as necessary)
seq = iaa.Sequential([
    iaa.Sometimes(0.6, [
        iaa.Multiply((1.2, 1.5)),  # Adjust brightness
        iaa.Fliplr(0.5),  # Horizontally flip 50% of the images
        iaa.Flipud(0.2),  # Vertically flip 20% of the images
        iaa.Affine(
            rotate=(-10, 10),  # Rotate between -10 and 10 degrees
            scale=(0.9, 1.1),  # Scale between 90% and 110%
            translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},  # Translate between -10% and 10%
            shear=(-10, 10)  # Shear between -10 and 10 degrees
        ),
        iaa.LinearContrast((0.75, 1.5)),  # Adjust contrast
    ])
])

# Load bounding boxes from a label file
def load_bounding_boxes(label_path, image_shape):
    bbs = []
    class_ids = []
    with open(label_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            # Read class ID and bounding box details
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            class_ids.append(int(class_id))
            # Convert from YOLO format (center x/y, width, height) to (x1, y1, x2, y2)
            x1 = (x_center - width / 2) * image_shape[1]
            y1 = (y_center - height / 2) * image_shape[0]
            x2 = (x_center + width / 2) * image_shape[1]
            y2 = (y_center + height / 2) * image_shape[0]
            bbs.append(BoundingBox(x1=x1, y1=y1, x2=x2, y2=y2))
    return BoundingBoxesOnImage(bbs, shape=image_shape), class_ids

# Save updated bounding boxes after augmentation
def save_bounding_boxes(label_path, bbs_aug, class_ids, image_shape):
    with open(label_path, 'w') as f:
        for i, bb in enumerate(bbs_aug.bounding_boxes):
            # Convert back to YOLO format (center x/y, width, height)
            x_center = (bb.x1 + bb.x2) / 2 / image_shape[1]
            y_center = (bb.y1 + bb.y2) / 2 / image_shape[0]
            width = (bb.x2 - bb.x1) / image_shape[1]
            height = (bb.y2 - bb.y1) / image_shape[0]
            class_id = class_ids[i]
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Process all images and labels in the folder
def augment_images_in_folder(image_folder, label_folder, output_image_folder, output_label_folder):
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    # Calculate the number of images to select (60% of the total)
    num_images_to_select = int(0.6 * len(image_files))
    
    # Randomly select 60% of the images
    selected_image_files = random.sample(image_files, num_images_to_select)
    
    for image_file in image_files:  # Iterate over all images to copy original + augment 60%
        image_path = os.path.join(image_folder, image_file)
        label_path = os.path.join(label_folder, image_file.replace('.jpg', '.txt').replace('.png', '.txt'))
        
        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Could not read image {image_file}")
            continue
        
        # Copy original image and label to the output folder
        shutil.copy(image_path, os.path.join(output_image_folder, image_file))
        shutil.copy(label_path, os.path.join(output_label_folder, image_file.replace('.jpg', '.txt').replace('.png', '.txt')))
        
        # Augment only selected images (60% of the total)
        if image_file in selected_image_files:
            # Load bounding boxes and class IDs from the corresponding label file
            bbs, class_ids = load_bounding_boxes(label_path, image.shape)

            # Apply augmentations
            image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

            # Remove bounding boxes fully outside the image and clip those partially outside
            bbs_aug = bbs_aug.remove_out_of_image().clip_out_of_image()

            # Save the augmented image
            output_image_path = os.path.join(output_image_folder, f"aug_{image_file}")
            cv2.imwrite(output_image_path, image_aug)

            # Save the updated bounding boxes and class IDs
            output_label_path = os.path.join(output_label_folder, f"aug_{image_file.replace('.jpg', '.txt').replace('.png', '.txt')}")
            save_bounding_boxes(output_label_path, bbs_aug, class_ids, image_aug.shape)

            print(f"Processed {image_file} and saved augmented data.")

# Example usage:
image_folder = "OGsplit/val_cropped/images"
label_folder = "OGsplit/val_cropped/labels"
output_image_folder = "OGSplit/val_augment/images"
output_label_folder = "OGSplit/val_augment/labels"

os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(output_label_folder, exist_ok=True)

augment_images_in_folder(image_folder, label_folder, output_image_folder, output_label_folder)


Processed B1_1_Image004_ICC_rotate_0.jpg and saved augmented data.
Processed B1_1_Image004_ICC_rotate_1.jpg and saved augmented data.
Processed B1_1_Image004_ICC_rotate_2.jpg and saved augmented data.
Processed B1_2_Image003_ICC_rotate_0.jpg and saved augmented data.
Processed B1_2_Image003_ICC_rotate_1.jpg and saved augmented data.
Processed B1_2_Image003_ICC_rotate_2.jpg and saved augmented data.
Processed B1_2_Image003_ICC_rotate_3.jpg and saved augmented data.
Processed B2_1_Image006_ICC_rotate_5.jpg and saved augmented data.
Processed B2_PTX5_0_10hz_Image001_ICC_0.jpg and saved augmented data.
Processed B2_PTX5_0_10hz_Image001_ICC_1.jpg and saved augmented data.
Processed B2_PTX5_0_10hz_Image001_ICC_3.jpg and saved augmented data.
Processed B2_PTX5_0_10hz_Image001_ICC_4.jpg and saved augmented data.
Processed B2_PTX5_0_10hz_Image001_ICC_5.jpg and saved augmented data.
Processed B3_1_Image001_ICC_0.jpg and saved augmented data.
Processed B3_1_Image001_ICC_2.jpg and saved augmented 